In [13]:
import pymssql
import pandas as pd
import configparser

In [14]:
config = configparser.ConfigParser()
config.read('db_config.ini')

db_host = config['database']['host']
db_user = config['database']['user']
db_password = config['database']['password']
db_name = config['database']['dbname']

sql_conn = pymssql.connect(db_host, db_user, db_password, db_name)

In [16]:
def get_data(query, columns = None):
    """Query in SQL STRING
    Columns = List of strings representing the columns for the df"""
    mycursor = sql_conn.cursor()
    mycursor.execute(query)
    result = mycursor.fetchall()
    data_list = list(result)
    if columns == None:
        df = pd.DataFrame(data_list)
        return df
    else:
        df = pd.DataFrame(data_list, columns= columns)
        return df
query_diagnosis_columns = """SELECT *
FROM syscolumns
WHERE id=OBJECT_ID('CDW_NEW.deid_uf.DiagnosisEventFact') """
diagnosis_columns = get_data(query_diagnosis_columns)[0].to_list()
query_ms= """SELECT * FROM CDW_NEW.deid_uf.DiagnosisEventFact
WHERE DiagnosisName LIKE '%multiple sclerosis%' """
ms_diagnosis = get_data(query_ms, diagnosis_columns)